## 1. Setup: Model Initialization
*Establishes the connection to the local Ollama instance and configures generation parameters (Temperature, Model).*

In [1]:
import requests
import json
from IPython.display import display, Markdown

# Configuration for VerbaView
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2"  # Change this if you pulled a different model like 'qwen2.5-coder'

def talk_to_verbaview(prompt):
    """
    Sends a prompt to the local VerbaView brain (Ollama) and returns the text response.
    """
    headers = {
        "Content-Type": "application/json"
    }
    
    # We use a very low temperature (0.1) because we want precise code, not creative poetry.
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "temperature": 0.1 
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, headers=headers, data=json.dumps(payload))
        response.raise_for_status() # Check for errors
        
        response_data = response.json()
        return response_data.get("response", "")
        
    except requests.exceptions.RequestException as e:
        return f"Error connecting to VerbaView Brain: {e}"

# --- TEST BLOCK ---
# Let's test if the brain is alive.
print("Connecting to VerbaView Engine...")
test_response = talk_to_verbaview("Are you ready to write some code? Answer with just 'Yes'.")
print(f"VerbaView says: {test_response}")

Connecting to VerbaView Engine...
VerbaView says: Yes.


## 2. Prompt Engineering Layer: The "Senior Dev" Persona
*Constructs high-fidelity system instructions to enforce modern UI standards (Tailwind CSS), strict output formatting, and error-free DOM structures.*

In [2]:
def get_verbaview_prompt(user_instruction):
    """
    Constructs the specific prompt that forces the LLM to behave like a UI engine.
    """
    
    system_role = """
    You are VerbaView, an expert Frontend AI capable of generating production-grade UI instantly.
    
    YOUR RULES:
    1. OUTPUT FORMAT: Return ONLY valid HTML code. Do NOT write "Here is the code" or markdown backticks (```html). Just the code.
    2. TECH STACK: You MUST use Tailwind CSS via CDN for styling.
       - Use this script in head: <script src="[https://cdn.tailwindcss.com](https://cdn.tailwindcss.com)"></script>
       - Use this link for Icons: <link rel="stylesheet" href="[https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css](https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css)" />
    3. DESIGN STANDARDS:
       - Use modern UI principles: extensive whitespace, rounded corners (rounded-xl, rounded-2xl), nice shadows (shadow-lg), and gradients.
       - Use 'Inter' or 'Roboto' Google Fonts.
       - Make it fully responsive.
    4. CONTENT: Use placeholder images (like '[https://placehold.co/600x400](https://placehold.co/600x400)') if needed.
    5. SINGLE FILE: Put all CSS (in <style> if custom needed) and JS (in <script>) inside the one HTML string.
    """
    
    # We combine the system role and the user's specific request
    full_prompt = f"{system_role}\n\nUSER REQUEST: {user_instruction}\n\nGENERATE HTML:"
    return full_prompt

def generate_ui_code(user_instruction):
    """
    Orchestrates the flow: Prompt -> Model -> Raw Code
    """
    print(f"VerbaView is thinking about: '{user_instruction}'...")
    
    # 1. Build the prompt
    prompt = get_verbaview_prompt(user_instruction)
    
    # 2. Get response from Ollama
    raw_response = talk_to_verbaview(prompt)
    
    # 3. Clean up potential Markdown (Llama sometimes adds ```html anyway)
    clean_code = raw_response.replace("```html", "").replace("```", "").strip()
    
    return clean_code

# --- TEST BLOCK ---
# Let's see if it generates HTML text (We won't render it yet, just check the text)
print("Testing Prompt Engineering...")
code_output = generate_ui_code("Create a modern pricing card with 3 tiers: Basic, Pro, Enterprise. Make the Pro card stand out with a purple gradient.")

# Print the first 500 chars to verify it looks like HTML
print("\n--- SNIPPET OF GENERATED CODE ---")
print(code_output[:500] + "...")

Testing Prompt Engineering...
VerbaView is thinking about: 'Create a modern pricing card with 3 tiers: Basic, Pro, Enterprise. Make the Pro card stand out with a purple gradient.'...

--- SNIPPET OF GENERATED CODE ---
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Pricing Card</title>
    <link rel="stylesheet" href="https://cdn.tailwindcss.com">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        .basic {
            background-color: #f7f7f7;
            border-radius: 0.5rem;
            padding: 1.5rem 2.5rem;
            box...


## 3. Rendering Engine: Virtual DOM Visualization
*Converts the generated raw string into a live HTML file and renders it within a sandboxed IFrame to prevent CSS conflicts with the Jupyter environment.*

In [3]:
from IPython.display import IFrame, display
import os

def render_interface(html_code, filename="verbaview_preview.html"):
    """
    Saves the string to a real HTML file and displays it in an interactive IFrame.
    """
    # 1. Save the code to a physical file (Good for debugging)
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_code)
    
    print(f"✨ Interface generated! Saved to: {filename}")
    print("rendering preview below...")
    
    # 2. Render the IFrame
    # We use width=100% and height=600px to simulate a desktop view
    display(IFrame(src=filename, width="100%", height="600"))

# --- ACTION BLOCK ---
# Let's visualize the code we generated in the previous step!
# (Ensure you ran the previous cell so 'code_output' exists)

if 'code_output' in locals() and len(code_output) > 10:
    render_interface(code_output)
else:
    print("No code found. Please run the generation cell above first!")

✨ Interface generated! Saved to: verbaview_preview.html
rendering preview below...


## 3.1 Middleware: Code Sanitization & Validation
*A post-processing layer that automatically corrects common LLM hallucinations (e.g., incorrect CDN tags) to ensure successful rendering.*

In [4]:
def sanitize_code(raw_html):
    """
    Fixes common AI errors before rendering.
    """
    print("🧹 Sanitizing code...")
    
    # FIX 1: Correct the Tailwind CDN
    # The model often confuses <link> and <script> for Tailwind
    if '<link rel="stylesheet" href="https://cdn.tailwindcss.com">' in raw_html:
        print("   -> Fixed incorrect Tailwind import (Link -> Script)")
        raw_html = raw_html.replace(
            '<link rel="stylesheet" href="https://cdn.tailwindcss.com">',
            '<script src="https://cdn.tailwindcss.com"></script>'
        )
        
    # FIX 2: Ensure it has the script even if missing
    if "cdn.tailwindcss.com" not in raw_html:
        print("   -> Injected missing Tailwind Script")
        head_end = raw_html.find("</head>")
        if head_end != -1:
            raw_html = raw_html[:head_end] + '<script src="https://cdn.tailwindcss.com"></script>' + raw_html[head_end:]
            
    return raw_html

# --- ACTION BLOCK ---
# 1. Get the previous bad output
dirty_code = code_output 

# 2. Clean it
clean_code = sanitize_code(dirty_code)

# 3. Render the fixed version
render_interface(clean_code, filename="verbaview_fixed.html")

🧹 Sanitizing code...
   -> Fixed incorrect Tailwind import (Link -> Script)
✨ Interface generated! Saved to: verbaview_fixed.html
rendering preview below...


## 4. State Management: The Iterative Refinement Loop
*Implements a context-aware update system that feeds the existing DOM state back into the LLM, enabling conversational UI updates (e.g., "Make the button blue").*

In [5]:
def update_ui_code(previous_code, user_instruction):
    """
    Takes existing code + a change request -> Returns new code.
    """
    print(f"VerbaView is refining the design based on: '{user_instruction}'...")
    
    # We create a specific prompt for UPDATING, not creating from scratch.
    prompt = f"""
    You are VerbaView. Here is an existing HTML file:
    
    {previous_code}
    
    USER INSTRUCTION: {user_instruction}
    
    YOUR TASK:
    1. Update the code exactly as requested.
    2. Keep all other styles (Tailwind) and structure intact.
    3. Return ONLY the full, valid HTML string.
    """
    
    # 1. Send to Ollama
    raw_response = talk_to_verbaview(prompt)
    
    # 2. Clean the markdown tags
    new_code = raw_response.replace("```html", "").replace("```", "").strip()
    
    # 3. Sanitize it (Run our self-healing layer again just in case!)
    final_code = sanitize_code(new_code)
    
    return final_code

# --- ACTION BLOCK ---
# Let's test the loop!
# We will take 'clean_code' (from the previous step) and change the button color.

if 'clean_code' in locals():
    # Example: The button is currently Orange. Let's make it Indigo.
    updated_code = update_ui_code(clean_code, "Change the Sign Up button to a deep Indigo color and make it full width.")
    
    # Render the new version
    render_interface(updated_code, filename="verbaview_v2.html")
    
    # Update our variable so we can keep chaining changes!
    clean_code = updated_code
else:
    print("No previous code found. Run Phase 3 first.")

VerbaView is refining the design based on: 'Change the Sign Up button to a deep Indigo color and make it full width.'...
🧹 Sanitizing code...
✨ Interface generated! Saved to: verbaview_v2.html
rendering preview below...


## 5. The Application Layer: Interactive Dashboard
*Integrates `ipywidgets` to create a user-friendly frontend for the model, simulating a real-world SaaS product experience within the notebook.*

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global variable to hold state (memory)
current_ui_code = ""

# --- WIDGETS LAYOUT ---
title_label = widgets.HTML("<h2>🎨 VerbaView Studio</h2>")
instruction_input = widgets.Textarea(
    placeholder="Describe your UI (e.g., 'Make a login page for a coffee shop')...",
    layout=widgets.Layout(width='100%', height='80px')
)
generate_btn = widgets.Button(
    description="✨ Generate UI",
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='200px')
)
status_output = widgets.Output() # Area to show print messages
preview_output = widgets.Output() # Area to show the website

# --- EVENT HANDLER ---
def on_click_generate(b):
    global current_ui_code
    
    # 1. Get user input
    user_text = instruction_input.value
    if not user_text:
        return
    
    with status_output:
        clear_output()
        print(f"🧠 VerbaView is working on: '{user_text}'...")
        generate_btn.description = "⏳ Thinking..."
        generate_btn.disabled = True
        
    # 2. Decide: Is this a NEW request or an UPDATE?
    try:
        if current_ui_code == "":
            # First time -> Generate from scratch
            raw_code = generate_ui_code(user_text)
        else:
            # Has history -> Update existing
            raw_code = update_ui_code(current_ui_code, user_text)
            
        # 3. Sanitize
        final_code = sanitize_code(raw_code)
        
        # 4. Update State
        current_ui_code = final_code
        
        # 5. Render
        with preview_output:
            clear_output()
            # Save to file to ensure IFrame picks it up
            render_interface(final_code, filename="verbaview_live.html")
            
        with status_output:
            print("✅ Done! Preview updated below.")
            
    except Exception as e:
        with status_output:
            print(f"❌ Error: {str(e)}")
            
    finally:
        generate_btn.description = "✨ Generate / Update"
        generate_btn.disabled = False

# Connect button to function
generate_btn.on_click(on_click_generate)

# --- DISPLAY THE APP ---
dashboard = widgets.VBox([
    title_label,
    instruction_input,
    generate_btn,
    status_output,
    widgets.HTML("<hr>"), # Divider
    preview_output
])

display(dashboard)